In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import platform
import tensorflow as tf


print('python_version',platform.python_version())
print('tensorflow',tf.__version__)

In [ ]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np

mnist=keras.datasets.mnist

In [ ]:
class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = {'batch': [], 'epoch': []}
        self.accuracy = {'batch': [], 'epoch': []}
        self.val_loss = {'batch': [], 'epoch': []}
        self.val_acc = {'batch': [], 'epoch': []}

    def on_batch_end(self, batch, logs={}):
        self.losses['batch'].append(logs.get('loss'))
        self.accuracy['batch'].append(logs.get('acc'))
        self.val_loss['batch'].append(logs.get('val_loss'))
        self.val_acc['batch'].append(logs.get('val_acc'))

    def on_epoch_end(self, batch, logs={}):
        self.losses['epoch'].append(logs.get('loss'))
        self.accuracy['epoch'].append(logs.get('acc'))
        self.val_loss['epoch'].append(logs.get('val_loss'))
        self.val_acc['epoch'].append(logs.get('val_acc'))

    def loss_plot(self, loss_type):
        iters = range(len(self.losses[loss_type]))
        #创建一个图
        plt.figure()
        # acc
        plt.plot(iters, self.accuracy[loss_type], 'r', label='train acc')#plt.plot(x,y)，这个将数据画成曲线
        # loss
        plt.plot(iters, self.losses[loss_type], 'g', label='train loss')
        if loss_type == 'epoch':
            # val_acc
            plt.plot(iters, self.val_acc[loss_type], 'b', label='val acc')
            # val_loss
            plt.plot(iters, self.val_loss[loss_type], 'k', label='val loss')
        plt.grid(True)#设置网格形式
        plt.xlabel(loss_type)
        plt.ylabel('acc-loss')#给x，y轴加注释
        plt.legend(loc="upper right")#设置图例显示位置
        plt.show()

#创建一个实例LossHistory
history = LossHistory()

In [ ]:
# 获取 数据 
def get_train_val(mnist_path):
    # mnist下载地址：https://storage.googleapis.com/tensorflow/tf-keras-datasets/mnist.npz
    (train_images, train_labels), (test_images, test_labels) = mnist.load_data(mnist_path)
    # 打印 训练数据和测试数据的长度
    print("train_images nums:{}".format(len(train_images)))
    print("test_images nums:{}".format(len(test_images)))
    return train_images, train_labels, test_images, test_labels
 
# 展示 图片 数据
def show_mnist(images,labels):
    for i in range(25):
        plt.subplot(5,5,i+1)
        plt.xticks([])
        plt.yticks([ ])
        plt.grid(False)
        plt.imshow(images[i],cmap=plt.cm.gray)
        plt.xlabel(str(labels[i]))
    plt.show()

# one_hot 编码，对标签进行 one_hot
def one_hot(labels):
    onehot_labels=np.zeros(shape=[len(labels),10])
    for i in range(len(labels)):
        index=labels[i]
        onehot_labels[i][index]=1
    return onehot_labels
 
# 搭建 简单 网络 
def mnist_net(input_shape):
    '''
    构建一个简单的全连接层网络模型：
    输入层为28x28=784个输入节点
    隐藏层120个节点
    输出层10个节点
    :param input_shape: 指定输入维度
    :return:
    '''
    # 使用 顺序模型
    model = keras.Sequential()
    # 输入层  input_shape 输入的 大小
    model.add(keras.layers.Flatten(input_shape=input_shape))
    #隐含层  units 神经元的个数120，activation 激活函数 位 rule
    model.add(keras.layers.Dense(units=120, activation=tf.nn.relu)) 
    #输出层 units 神经元的个数10（10个数字），activation 激活函数 位 softmax
    model.add(keras.layers.Dense(units=10, activation=tf.nn.softmax))
    return model
 
# 搭建 CNN 
def mnist_cnn(input_shape):
    '''
    构建一个CNN网络模型
    :param input_shape: 指定输入维度
    :return:
    '''
    # 使用 顺序模型
    model=keras.Sequential()
    # 卷积层 ，kernel_size卷积核大小，activation ，激活函数 ，input_shape 输入大小
    model.add(keras.layers.Conv2D(filters=32,kernel_size = 5,strides = (1,1),
                                  padding = 'same',activation = tf.nn.relu,input_shape = input_shape))
    # 池化层 pool_size 池化大小
    model.add(keras.layers.MaxPool2D(pool_size=(2,2), strides = (2,2), padding = 'valid'))
     # 卷积层 
    model.add(keras.layers.Conv2D(filters=64,kernel_size = 3,strides = (1,1),padding = 'same',activation = tf.nn.relu))
    # 池化层
    model.add(keras.layers.MaxPool2D(pool_size=(2,2), strides = (2,2), padding = 'valid'))
    # 需要丢弃的输入比例。可以防止过拟合
    model.add(keras.layers.Dropout(0.25))
    # 展平一个张量
    model.add(keras.layers.Flatten())
    #全连接层
    model.add(keras.layers.Dense(units=128,activation = tf.nn.relu))
    model.add(keras.layers.Dropout(0.5))
    # 全连接层 输出
    model.add(keras.layers.Dense(units=10,activation = tf.nn.softmax))
    return model

# 训练 模型
def trian_model(train_images,train_labels,test_images,test_labels):
    # re-scale to 0~1.0之间，把数据 压缩到 0-1 之间
    train_images=train_images/255.0
    test_images=test_images/255.0
    # mnist数据转换为四维
    train_images=np.expand_dims(train_images,axis = 3)
    test_images=np.expand_dims(test_images,axis = 3)
    print("train_images :{}".format(train_images.shape))
    print("test_images :{}".format(test_images.shape))
    # 标签 one—hot 
    train_labels=one_hot(train_labels)
    test_labels=one_hot(test_labels)
 
    # 建立模型
    model = mnist_net(input_shape=(28,28,1)) # 简单 网络
    #model=mnist_cnn(input_shape=(28,28,1))  # CNN 网络
    #配置训练模型。optimizer优化器，loss损失函数
    model.compile(optimizer=tf.optimizers.Adam(),loss="categorical_crossentropy",metrics=['accuracy'])
    # 开始 训练 模型 
    model.fit(x=train_images,y=train_labels,epochs=5,validation_data=(test_images,test_labels),callbacks=[history])
    
    # 测试模式 返回 损失值和准确率。
    test_loss,test_acc=model.evaluate(x=test_images,y=test_labels)
    print("Test Accuracy %.2f"%test_acc)
 
    # 开始预测
    cnt=0
    predictions=model.predict(test_images)
    for i in range(len(test_images)):
        target=np.argmax(predictions[i])
        label=np.argmax(test_labels[i])
        if target==label:
            cnt +=1
    # 准确率
    print("True prediction of total : %.2f"%(cnt/len(test_images)))
    # 保存 模型
    model.save('mnist-model.h5')
    


In [ ]:
mnist_path = '/kaggle/input/dataet/mnist.npz'
# 获取 数据 
train_images, train_labels, test_images, test_labels=get_train_val(mnist_path)
# 展示 数据 
show_mnist(train_images, train_labels)
# 训练 和 预测
trian_model(train_images, train_labels, test_images, test_labels)
# 绘制acc-loss曲线
history.loss_plot('epoch')

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Convolution2D, MaxPooling2D
from tensorflow.keras.datasets import mnist

np.random.seed(123)  # for reproducibility

mnist_path = '/kaggle/input/dataet/mnist.npz'

In [ ]:
def load_data(mnist_path):
    """Loads the MNIST dataset.
    # Arguments
        path: path where to cache the dataset locally
            (relative to ~/.keras/datasets).
    # Returns
        Tuple of Numpy arrays: `(x_train, y_train), (x_test, y_test)`.
    """
    #path = get_file(path,
    #                origin='https://s3.amazonaws.com/img-datasets/mnist.npz',
    #                file_hash='8a61469f7ea1b51cbae51d4f78837e45')
    f = np.load(path)
    x_train, y_train = f['x_train'], f['y_train']
    x_test, y_test = f['x_test'], f['y_test']
    f.close()
    return (x_train, y_train), (x_test, y_test)
